In [1]:
import grpc
from tensorflow import make_tensor_proto
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_log_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
from tensorflow_serving.apis import get_model_metadata_pb2

2022-05-24 06:39:50.937457: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-24 06:39:50.937482: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import cv2
import numpy as np
from PIL import Image
from patchify import patchify

PATCH_SIZE = 480

def patch_image(image):

    instances = []

    if type(image) == str:
        image = cv2.imread(image)
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    size_x = (image.shape[1] // PATCH_SIZE) * PATCH_SIZE  # get width to nearest size divisible by patch size
    size_y = (image.shape[0] // PATCH_SIZE) * PATCH_SIZE  # get height to nearest size divisible by patch size

    image = Image.fromarray(image)

    # Crop original image to size divisible by patch size from top left corner
    image = np.array(image.crop((0, 0, size_x, size_y)))

    # Extract patches from each image, step=patch_size means no overlap
    patch_img = patchify(image, (PATCH_SIZE, PATCH_SIZE, 3), step=PATCH_SIZE)

    # iterate over vertical patch axis
    for j in range(patch_img.shape[0]):
        # iterate over horizontal patch axis
        for k in range(patch_img.shape[1]):
            # patches are located like a grid. use (j, k) indices to extract single patched image
            single_patch_img = patch_img[j, k]

            # Drop extra extra dimension from patchify
            instances.append(np.squeeze(single_patch_img))

    return instances, (int(size_x/PATCH_SIZE), int(size_y/PATCH_SIZE))

In [3]:
def depatchify(patches_arr, size, p_size=PATCH_SIZE):
    patches_np = np.array(patches_arr)

    w, h = size

    reshaped = patches_np.reshape(w, h, p_size, p_size, 3)

    bag = []

    for subindex in range(reshaped.shape[0]):
        line = np.concatenate(reshaped[subindex],axis=1)
        bag.append(line)

    image_to_reshape = np.array(bag)



    return image_to_reshape.reshape(w*p_size, h*p_size, 3)

In [4]:
from enum import Enum
class MaskColorMap(Enum):
    Urban = (0, 255, 255) # Cyan
    Agriculture = (255, 255, 0) # Amarillo
    Rangeland = (255, 0, 255) # Morado
    Forest = (0, 255, 0) # Verde
    Water = (0, 0, 255) # Azul
    Barren = (255, 255, 255) # Blanco
    Uknown = (0,0,0) # Negro

In [5]:
from tensorflow.keras.backend import flatten, sum
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

def jaccard_index(y_true, y_pred):
    y_true_f = flatten(y_true)
    y_pred_f = flatten(y_pred)
    intersection = sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (sum(y_true_f) + sum(y_pred_f) - intersection + 1.0)

In [6]:
def rgb_encode_mask(mask):
    # initialize rgb image with equal spatial resolution
    rgb_encode_image = np.zeros((mask.shape[0], mask.shape[1], 3))

    # iterate over MaskColorMap
    for j, cls in enumerate(MaskColorMap):
        # convert single integer channel to RGB channels
        rgb_encode_image[(mask == j)] = np.array(cls.value) / 255.
    return rgb_encode_image

In [7]:
def visualize(image):
    img = Image.fromarray(image)
    img.show()

In [5]:
from tensorflow.keras.models import load_model

MODEL_FILE = 'tf2-ecrop.h5'

model = load_model(MODEL_FILE, custom_objects={'jaccard_index': jaccard_index})

2022-05-19 08:02:16.573698: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-19 08:02:16.574494: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 08:02:16.574839: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-05-19 08:02:16.575174: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-05-19 08:02:16.575518: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [6]:
from tensorflow.keras.models import save_model

save_model(model, 'ecrop-main-serving')

2022-05-19 08:02:18.988192: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ecrop-main-serving/assets


In [38]:
options = [('grpc.max_receive_message_length', 100*1024*1024)]

channel = grpc.insecure_channel('localhost:8500',options=options)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

MODEL = 'ecrop-main-serving'
VERSION = 1

INPUT_LAYER = 'input_1'
OUTPUT_LAYER = 'conv2d_18'

request = predict_pb2.PredictRequest()
request.model_spec.name = MODEL
request.model_spec.version.value = VERSION


TypeError: __init__() got an unexpected keyword argument 'options'

In [33]:
patches, size = patch_image('input/37586_sat.jpg')

img = patches[0]

In [34]:
sendable = np.expand_dims(img, 0)

request.inputs[INPUT_LAYER].CopyFrom(make_tensor_proto(sendable, shape=sendable.shape, dtype=float))

In [35]:
sendable.shape

(1, 480, 480, 3)

In [36]:
sendable.dtype

dtype('uint8')

In [37]:
result = stub.Predict(request, 10)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.RESOURCE_EXHAUSTED
	details = "Received message larger than max (6451291 vs. 4194304)"
	debug_error_string = "{"created":"@1652949320.111860436","description":"Error received from peer ipv6:[::1]:8500","file":"src/core/lib/surface/call.cc","file_line":952,"grpc_message":"Received message larger than max (6451291 vs. 4194304)","grpc_status":8}"
>

In [ ]:
pred = model.predict(sendable)
pred = np.squeeze(pred)
pred = np.argmax(pred, axis=-1)
pred = rgb_encode_mask(pred)
pred = pred*255

p = np.array(pred, dtype=np.uint8)

visualize(p)

In [8]:
from tensorflow.keras.models import load_model

model = load_model('resnetv2.h5', custom_objects={'jaccard_index': jaccard_index})


2022-05-24 06:50:11.675926: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 06:50:11.676788: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /media/DATA1/shared/PBL6/env/lib/python3.8/site-packages/cv2/../../lib64:
2022-05-24 06:50:11.677221: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /media/DATA1/shared/PBL6/env/lib/python3.8/site-packages/cv2/../../lib64:
2022-05-24 06:50:11.677619: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.1

In [9]:
model.save('ecrop-main-serving', save_format='tf',  )

2022-05-24 06:50:32.903480: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ecrop-main-serving/assets


In [10]:
out = model.predict(sendable.astype(np.uint8))

NameError: name 'sendable' is not defined